In [4]:
#Created By Gareth Hutchins April 28th 2020
#Process Kaggle COVID19 Dataset and sent it to Magellan Text Mining to get out of the box cartridges
#Dataset can be found here https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge

import os
import json
import requests
import lxml.etree #First install in Terminal using pip install lxml

#We need to figure out the path to the notebool
notebook_path = os.path.abspath("COVID19 Kaggle To MTM.ipynb")
print("Notebook Path " + notebook_path)
#Now change that so we have replace the notebook name
#Set the directory
notebook_path = notebook_path.replace("COVID19 Kaggle To MTM.ipynb","")
print("Parent Directory " + notebook_path)
#DirectoryName = notebook_path + "COVID19Data/biorxiv_medrxiv/"
#DirectoryName = notebook_path + "COVID19Data/comm_use_subset/"
DirectoryName = notebook_path + "COVID19Data/custom_license/"
#DirectoryName = notebook_path + "COVID19Data/noncomm_use_subset/"

print("Using Directory " + DirectoryName)
#set the text mining uri
uri = "

Directory = os.fsencode(DirectoryName)
def extract_values(obj, key):
    #Pull all values of specified key from nested JSON
    arr = []

    def extract(obj, arr, key):
        #Recursively search for values of key in JSON tree.
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results
#Loop throug each file
for file in os.listdir(Directory) :
    #Get the filename
    filename = os.fsdecode(file)
    #Load the JSON file
     #Set the lists
    DISEASES = []
    SYMPTOMS = []
    DRUGS = []
    Locations = []
    ComplexConcepts =[]
    SimpleConcepts = []
    Categories = []
    Epidemics = []
    with open(DirectoryName + filename) as json_file:
        #Load the json file
        data = json.load(json_file)
        #Look for all of the text value
        text = extract_values(data,'text')
        tcount =0
        for t in text:
            postheaders = {}
            postheaders['Content-Type'] = "application/xml"
            #Create the xml post add the text
            xml = "<?xml version=" + '"' + "1.0" + '"'+ " encoding=" + '"' + "UTF-8" + '"'+ " ?><Nserver><ResultEncoding>UTF-8</ResultEncoding><TextID>COVID</TextID><NSTEIN_Text>"+ t + "</NSTEIN_Text><LanguageID>ENGLISH</LanguageID><Methods><nconceptextractor><ExcludeEntities/><SimpleConcepts></SimpleConcepts><ComplexConcepts><RelevancyLevel>FIRST</RelevancyLevel></ComplexConcepts></nconceptextractor><nfinder><nfExtract><Cartridges><Cartridge>GL</Cartridge><Cartridge>DISEASES</Cartridge><Cartridge>DRUGS</Cartridge><Cartridge>SYMPTOMS</Cartridge></Cartridges></nfExtract><nfFullTextSearch><Cartridges><Cartridge>Epidemics</Cartridge><Cartridge>DISEASES</Cartridge><Cartridge>DRUGS</Cartridge><Cartridge>SYMPTOMS</Cartridge><Cartridge>GL</Cartridge></Cartridges></nfFullTextSearch></nfinder><nsentiment Name=" + '"' +"RequestName" + '"' + "><Name>Nstein</Name></nsentiment><ncategorizer><KnowledgeBase><KBid>IPTC</KBid><NumberOfCategories>10</NumberOfCategories></KnowledgeBase><KnowledgeBase><KBid>MeSHLite</KBid><NumberOfCategories>10</NumberOfCategories></KnowledgeBase></ncategorizer></Methods></Nserver>"
            r = requests.post(uri,data=xml.encode('UTF-8'),headers=postheaders)
            if r.status_code != 200:
                continue
            #Get the results
            #sentiment
            root = lxml.etree.fromstring(r.content)
            sentiment = root.find('Results/nsentiment/DocumentLevel/Tone')
            if hasattr(sentiment,'text'):
                sentiment = sentiment.text
            #Get the Concepts
            #Complex
            complex_concepts = root.findall('Results/nconceptextractor/ComplexConcepts/Concept')
            for cc in complex_concepts:
                if hasattr(cc,'text'):
                    ComplexConcepts.append(cc.text)
            #Simple
            simple_concepts = root.findall('Results/nconceptextractor/SimpleConcepts/Concept')
            for sc in simple_concepts:
                if hasattr(sc,'text'):
                    SimpleConcepts.append(sc.text)
            #Get the Categories
            categories = root.findall('Results/ncategorizer/KnowledgeBase/Categories/Category')
            for c in categories:
                if hasattr(c,'text'):
                    Categories.append(c.text)
            #Get the Entities
            FTResults = root.findall('Results/nfinder/nfFullTextSearch/ExtractedTerm')
            #Loop through all results
            for ft in FTResults:
                if 'CartridgeID' in ft.attrib:
                    ftType = ft.attrib['CartridgeID']
                    #Get the main term
                    for MTchild in ft:
                        if MTchild.tag == 'MainTerm':
                            MainTerm = MTchild.text
                            MT = ft.find('MainTerm')
                            if ftType == "DISEASES":
                                if MainTerm not in DISEASES:
                                    DISEASES.append(MainTerm)
                                if ftType == "SYMPTOMS":
                                    if MainTerm not in SYMPTOMS:
                                        SYMPTOMS.append(MainTerm)
                                if ftType == "DRUGS":
                                    if MainTerm not in DRUGS:
                                        DRUGS.append(MainTerm)
                                if ftType == "GL":
                                    if MainTerm not in Locations:
                                        Locations.append(MainTerm)
                                if ftType == "Epidemics":
                                    if MainTerm not in Epidemics:
                                        Epidemics.append(MainTerm)
                        #Now get the Sub Terms
                            for STchild in MTchild:
                                if ftType == "DISEASES":
                                    if STchild.text not in DISEASES:
                                        DISEASES.append(STchild.text)
                                if ftType == "SYMPTOMS":
                                    if STchild.text not in SYMPTOMS:
                                        SYMPTOMS.append(STchild.text)
                                if ftType == "DRUGS":
                                    if STchild.text not in DRUGS:
                                        DRUGS.append(STchild.text)
                                if ftType == "GL":
                                    if STchild.text not in Locations:
                                        Locations.append(STchild.text)
                                if ftType == "Epidemics":
                                    if STchild.text not in Epidemics:
                                        Epidemics.append(STchild.text)
            NFResults = root.findall('Results/nfinder/nfExtract/ExtractedTerm')
            #Loop through all results
            for nf in FTResults:
                if 'CartridgeID' in nf.attrib:
                    nfType = nf.attrib['CartridgeID']
                    #Get the main term
                    for MTchild in nf:
                        if MTchild.tag == 'MainTerm':
                            MainTerm = MTchild.text
                            MT = nf.find('MainTerm')
                            if nfType == "DISEASES":
                                if MainTerm not in DISEASES:
                                    DISEASES.append(MainTerm)
                            if nfType == "SYMPTOMS":
                                if MainTerm not in SYMPTOMS:
                                    SYMPTOMS.append(MainTerm)
                            if nfType == "DRUGS":
                                if MainTerm not in DRUGS:
                                    DRUGS.append(MainTerm)
                            if nfType == "GL":
                                if MainTerm not in Locations:
                                    Locations.append(MainTerm)
                            if nfType == "Epidemics":
                                if MainTerm not in Epidemics:
                                    Epidemics.append(MainTerm)
                        #Now get the Sub Terms
                        for STchild in MTchild:
                            if nfType == "DISEASES":
                                if STchild.text not in DISEASES:
                                    DISEASES.append(STchild.text)
                            if nfType == "SYMPTOMS":
                                if STchild.text not in SYMPTOMS:
                                    SYMPTOMS.append(STchild.text)
                            if nfType == "DRUGS":
                                if STchild.text not in DRUGS:
                                    DRUGS.append(STchild.text)
                            if nfType == "GL":
                                if STchild.text not in Locations:
                                    Locations.append(STchild.text)
                            if nfType == "Epidemics":
                                if MainTerm not in Epidemics:
                                    Epidemics.append(STchild.text)
            #Write the files out
            #Change the path to be output
            output_path = notebook_path + "Output/"
            #text id
            textid=open(notebook_path + "TextID.txt","a+")
            textid.write(str(tcount)+"_" + filename + "\n")
            textid.close
            print("Count " + str(tcount))
            
            #sentiment
            sent=open(notebook_path + "Sentiment.txt","a+")
            sent.write(str(tcount)+"_" + filename + ";" + str(sentiment) + "\n")
            sent.close
            #Write the diseases to file
            txtdis=open(notebook_path + "DISEASES.txt","a+")
            for D in DISEASES:
                txtdis.write(str(tcount)+"_" + filename + ";" + str(D) + ";" + "\n")
            txtdis.close
            #Write the Drugs to file
            txtdrugs=open(notebook_path + "DRUGS.txt","a+")
            for dr in DRUGS:
                txtdrugs.write(str(tcount)+"_" + filename + ";" + str(dr) + ";" + "\n")
            txtdrugs.close
            txtsymp=open(notebook_path + "Symptoms.txt","a+")
            for s in SYMPTOMS:
                txtsymp.write(str(tcount)+"_" + filename + ";" + str(s) + ";" + "\n")
            txtsymp.close
            ##Concepts
            txtcc=open(notebook_path + "ComplexConcepts.txt","a+")
            for cc in ComplexConcepts:
                txtcc.write(str(tcount)+"_" + filename + ";" + str(cc) + ";" + "\n")
            txtcc.close
            txtsc=open(notebook_path + "SimpleConcepts.txt","a+")
            for sc in SimpleConcepts:
                txtsc.write(str(tcount)+"_" + filename + ";" + str(sc) + ";" + "\n")
            txtsc.close
            #Locations
            txtL=open(notebook_path + "Locations.txt","a+")
            for l in Locations:
                txtL.write(str(tcount)+"_" + filename + ";" + str(l) + ";" + "\n")
            txtL.close
            #Categories
            txtcats=open(notebook_path + "Categories.txt","a+")
            for cats in Categories:
                txtcats.write(str(tcount)+"_" + filename + ";" + str(cats) + ";" + "\n")
            txtcats.close
            
            #Epidemics
            txteps=open(notebook_path + "Epidemics.txt","a+")
            for eps in Epidemics:
                txteps.write(str(tcount)+"_" + filename + ";" + str(eps) + ";" + "\n")
            txteps.close
            
            #Increase the counts
            tcount = tcount + 1
    continue
print("Done")



Notebook Path /home/{user}/notebooks/COVID19 Kaggle/COVID19 Kaggle To MTM.ipynb
Parent Directory /home/{user}/notebooks/COVID19 Kaggle/
Using Directory /home/{user}/notebooks/COVID19 Kaggle/COVID19Data/custom_license/
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 75
Count 76
Count 77
Count 78
Count 79
Count 80
Count 81
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 

Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Cou

Count 80
Count 81
Count 82
Count 83
Count 84
Count 85
Count 86
Count 87
Count 88
Count 89
Count 90
Count 91
Count 92
Count 93
Count 94
Count 95
Count 96
Count 97
Count 98
Count 99
Count 100
Count 101
Count 102
Count 103
Count 104
Count 105
Count 106
Count 107
Count 108
Count 109
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Co

Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 75
Count 76
Count 77
Count 78
Count 79
Count 80
Count 81
Count 82
Count 83
Count 84
Count 85
Count 86
Count 87
Count 88
Count 89
Count 90
Count 91
Count 92
Count 93
Count 94
Count 95
Count 96
Count 97
Count 98
Count 99
Count 100
C

Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 75
Count 76
Count 77
Count 78
Count 79
Count 80
Count 81
Count 82
Count 83
Count 84
Count 85
Count 86
Count 87
Count 88
Count 89
Count 90
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42
Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 75
Count 76
Count 77
Count 78
Co

...

Count 43
Count 44
Count 45
Count 46
Count 47
Count 48
Count 49
Count 50
Count 51
Count 52
Count 53
Count 54
Count 55
Count 56
Count 57
Count 58
Count 59
Count 60
Count 61
Count 62
Count 63
Count 64
Count 65
Count 66
Count 67
Count 68
Count 69
Count 70
Count 71
Count 72
Count 73
Count 74
Count 75
Count 76
Count 77
Count 78
Count 79
Count 80
Count 81
Count 82
Count 83
Count 84
Count 85
Count 86
Count 87
Count 88
Count 89
Count 90
Count 91
Count 92
Count 93
Count 94
Count 95
Count 96
Count 97
Count 98
Count 99
Count 100
Count 101
Count 102
Count 103
Count 104
Count 105
Count 106
Count 107
Count 108
Count 109
Count 110
Count 0
Count 1
Count 2
Count 3
Count 4
Count 5
Count 6
Count 7
Count 8
Count 9
Count 10
Count 11
Count 12
Count 13
Count 14
Count 15
Count 16
Count 17
Count 18
Count 19
Count 20
Count 21
Count 22
Count 23
Count 24
Count 25
Count 26
Count 27
Count 28
Count 29
Count 30
Count 31
Count 32
Count 33
Count 34
Count 35
Count 36
Count 37
Count 38
Count 39
Count 40
Count 41
Count 42


Count 729
Count 730
Count 731
Count 732
Count 733
Count 734
Count 735
Count 736
Count 737
Count 738
Count 739
Count 740
Count 741
Count 742
Count 743
Count 744
Count 745
Count 746
Count 747
Count 748
Count 749
Count 750
Count 751
Count 752
Count 753
Count 754
Count 755
Count 756
Count 757
Count 758
Count 759
Count 760
Count 761
Count 762
Count 763
Count 764
Count 765
Count 766
Count 767
Count 768
Count 769
Count 770
Count 771
Count 772
Count 773
Count 774
Count 775
Count 776
Count 777
Count 778
Count 779
Count 780
Count 781
Count 782
Count 783
Count 784
Count 785
Count 786
Count 787
Count 788
Count 789
Count 790
Count 791
Count 792
Count 793
Count 794
Count 795
Count 796
Count 797
Count 798
Count 799
Count 800
Count 801
Count 802
Count 803
Count 804
Count 805
Count 806
Count 807
Count 808
Count 809
Count 810
Count 811
Count 812
Count 813
Count 814
Count 815
Count 816
Count 817
Count 818
Count 819
Count 820
Count 821
Count 822
Count 823
Count 824
Count 825
Count 826
Count 827
Count 828


Count 1499
Count 1500
Count 1501
Count 1502
Count 1503
Count 1504
Count 1505
Count 1506
Count 1507
Count 1508
Count 1509
Count 1510
Count 1511
Count 1512
Count 1513
Count 1514
Count 1515
Count 1516
Count 1517
Count 1518
Count 1519
Count 1520
Count 1521
Count 1522
Count 1523
Count 1524
Count 1525
Count 1526
Count 1527
Count 1528
Count 1529
Count 1530
Count 1531
Count 1532
Count 1533
Count 1534
Count 1535
Count 1536
Count 1537
Count 1538
Count 1539
Count 1540
Count 1541
Count 1542
Count 1543
Count 1544
Count 1545
Count 1546
Count 1547
Count 1548
Count 1549
Count 1550
Count 1551
Count 1552
Count 1553
Count 1554
Count 1555
Count 1556
Count 1557
Count 1558
Count 1559
Count 1560
Count 1561
Count 1562
Count 1563
Count 1564
Count 1565
Count 1566
Count 1567
Count 1568
Count 1569
Count 1570
Count 1571
Count 1572
Count 1573
Count 1574
Count 1575
Count 1576
Count 1577
Count 1578
Count 1579
Count 1580
Count 1581
Count 1582
Count 1583
Count 1584
Count 1585
Count 1586
Count 0
Count 1
Count 2
Count 3


Count 635
Count 636
Count 637
Count 638
Count 639
Count 640
Count 641
Count 642
Count 643
Count 644
Count 645
Count 646
Count 647
Count 648
Count 649
Count 650
Count 651
Count 652
Count 653
Count 654
Count 655
Count 656
Count 657
Count 658
Count 659
Count 660
Count 661
Count 662
Count 663
Count 664
Count 665


OSError: [Errno 28] No space left on device